In [1]:
from flask import Flask,render_template,request,redirect,flash,send_file
from werkzeug import secure_filename
import pandas as pd
import numpy as np
import os
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt
import io
import base64
import seaborn as sns
import glob
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler,LabelBinarizer
from sklearn.linear_model import LinearRegression,Lasso,Ridge,LogisticRegression
from sklearn.svm import SVR,SVC
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error,accuracy_score,precision_score,recall_score
from sklearn.metrics import r2_score,auc,roc_auc_score,f1_score,confusion_matrix
app = Flask(__name__)
import warnings
warnings.filterwarnings("ignore")
document = Document()

In [2]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return np.round(roc_auc_score(y_test, y_pred, average=average),4)

In [3]:
def r2_and_adjusted_r2(y_test,pred,x,y):
    r_squared = r2_score(y_test,pred)
    adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1)
    return r_squared,adjusted_r_squared

In [4]:
def plotting_and_saving_confusion_matrix(y_test,predict):
    try:
        ax = sns.heatmap(confusion_matrix(y_test,predict),annot=True)
        ax.set(xlabel='Predicted Label', ylabel='Actual Label')
        ax.set_title('Confusion Matrix')
        fig = ax.get_figure()
        fig.savefig("Confusion Matrix.PNG")
        plt.clf()
    except:
        pass

In [5]:
def reading_csv():
    try:
        files = glob.glob('*.csv')
        df = pd.read_csv(files[0])
    except:
        df = 'Please upload a CSV first'
    return df

In [6]:
@app.route('/')
def homepage():
    return render_template('welcome.html')

In [7]:
def scaling_type_value(scaling_parameter):
    if scaling_parameter =='MinMaxScaler':
        return MinMaxScaler()
    if scaling_parameter =='StandardScaler':
        return StandardScaler()
    if scaling_parameter == 'RobustScaler':
        return RobustScaler()

In [8]:
def model_regression(model,x,y,scaling):
    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    model.fit(x_train,y_train)
    predict = model.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test,predict))
    mse = mean_squared_error(y_test,predict)
    mae = mean_absolute_error(y_test,predict)
    r2,adj_r2 = r2_and_adjusted_r2(y_test,predict,x,y)
    return np.round(rmse,4),np.round(mse,4),np.round(mae,4),np.round(r2,4),np.round(adj_r2,4)

def model_classification(model,x,y,scaling):
    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    model.fit(x_train,y_train)
    predict_train = model.predict(x_train)
    predict = model.predict(x_test)
    acc_train = accuracy_score(y_train,predict_train)
    acc = accuracy_score(y_test,predict)
    precisionscore = precision_score(y_test,predict,average = None)
    recallscore = recall_score(y_test,predict,average = None)
    roc_score = multiclass_roc_auc_score(y_test,predict)
    f1 = f1_score(y_test,predict,average=None)
    plotting_and_saving_confusion_matrix(y_test,predict)
    return np.round(acc_train,4),np.round(acc,4),precisionscore,recallscore,roc_score,f1

In [9]:
def model_classification_grid_search_logistic(model,x,y,c,cv,scaling,regularization):
    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    param = {'C':np.arange(1,int(c)+1),'penalty':np.array(regularization)}
    grid = GridSearchCV(model,param_grid=param,cv=int(cv))
    grid.fit(x_train,y_train)
    predict_train = grid.predict(x_train)
    predict = grid.predict(x_test)
    acc_train = accuracy_score(y_train,predict_train)
    acc = accuracy_score(y_test,predict)
    precisionscore = precision_score(y_test,predict,average = None)
    recallscore = recall_score(y_test,predict,average = None) 
    roc_score = multiclass_roc_auc_score(y_test,predict)
    f1 = f1_score(y_test,predict,average=None)
    plotting_and_saving_confusion_matrix(y_test,predict)
    return np.round(acc_train,4),np.round(acc,4),precisionscore,recallscore,np.round(grid.best_score_,4),grid.best_params_,roc_score,f1

In [10]:
def model_classification_grid_search_decision_tree_regression(model,x,y,max_depth,min_samples_split,min_samples_leaf,
                                                             min_weight_fraction_leaf,max_leaf_nodes,cv,scaling):
    
    
    
    #,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,cv
    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    param = {'max_depth':np.arange(1,int(max_depth)+1),'min_samples_split':np.arange(0.1,float(min_samples_split)+0.1,0.1),
             'min_samples_leaf':np.arange(1,int(min_samples_leaf)+1), 'min_weight_fraction_leaf':np.arange(0.1,float(min_weight_fraction_leaf)+0.1,0.1)
             , 'max_leaf_nodes':np.arange(2,int(max_leaf_nodes)+1)}   
    grid = GridSearchCV(model,param_grid=param,cv=int(cv))
    grid.fit(x_train,y_train)
    predict = grid.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test,predict))
    mse = mean_squared_error(y_test,predict)
    mae = mean_absolute_error(y_test,predict)
    r2,adj_r2 = r2_and_adjusted_r2(y_test,predict,x,y)
    return np.round(rmse,4),np.round(mse,4),np.round(mae,4),np.round(grid.best_score_,4),grid.best_params_,np.round(r2,4),np.round(adj_r2,4)

In [11]:
def model_classification_grid_search_decision_tree_classification(model,x,y,max_depth,min_samples_split,min_samples_leaf,
                                                             min_weight_fraction_leaf,max_leaf_nodes,cv,scaling):

    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    
    param = {'max_depth':np.arange(1,int(max_depth)+1),'min_samples_split':np.arange(0.1,float(min_samples_split)+0.1,0.1),
             'min_samples_leaf':np.arange(1,int(min_samples_leaf)+1), 'min_weight_fraction_leaf':np.arange(0.1,float(min_weight_fraction_leaf)+0.1,0.1)
             , 'max_leaf_nodes':np.arange(2,int(max_leaf_nodes)+1)}   
    grid = GridSearchCV(model,param_grid=param,cv=int(cv))
    grid.fit(x_train,y_train)
    predict_train = grid.predict(x_train)
    predict = grid.predict(x_test)
    acc_train = accuracy_score(y_train,predict_train)
    acc = accuracy_score(y_test,predict)
    precisionscore = precision_score(y_test,predict,average = None)
    recallscore = recall_score(y_test,predict,average = None)  
    roc_score = multiclass_roc_auc_score(y_test,predict)
    f1 = f1_score(y_test,predict,average=None)
    plotting_and_saving_confusion_matrix(y_test,predict)
    return np.round(acc_train,4),np.round(acc,4),precisionscore,recallscore,np.round(grid.best_score_,4),grid.best_params_,roc_score,f1

In [12]:
def model_classification_grid_search_knn_regression(model,x,y,cv,neighbours,metric,scaling):
    
    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    param = {'n_neighbors':np.arange(1,int(neighbours)+1),'metric':metric}
    grid = GridSearchCV(model,param_grid=param,cv=int(cv))
    grid.fit(x_train,y_train)
    predict = grid.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test,predict))
    mse = mean_squared_error(y_test,predict)
    mae = mean_absolute_error(y_test,predict)
    r2,adj_r2 = r2_and_adjusted_r2(y_test,predict,x,y)

    return np.round(rmse,4),np.round(mse,4),np.round(mae,4),np.round(grid.best_score_,4),grid.best_params_,np.round(r2,4),np.round(adj_r2,4)

In [13]:
def model_classification_grid_search_knn_classification(model,x,y,cv,neighbours,metric,scaling):
    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    param = {'n_neighbors':np.arange(1,int(neighbours)+1),'metric':metric}
    grid = GridSearchCV(model,param_grid=param,cv=int(cv),scoring = 'accuracy')
    grid.fit(x_train,y_train)
    predict_train = grid.predict(x_train)
    predict = grid.predict(x_test)
    acc_train = accuracy_score(y_train,predict_train)
    acc = accuracy_score(y_test,predict)
    precisionscore = precision_score(y_test,predict,average = None)
    recallscore = recall_score(y_test,predict,average = None)  
    roc_score = multiclass_roc_auc_score(y_test,predict)
    f1 = f1_score(y_test,predict,average=None)
    plotting_and_saving_confusion_matrix(y_test,predict)
    return np.round(acc_train,4),np.round(acc,4),precisionscore,recallscore,np.round(grid.best_score_,4),grid.best_params_,roc_score,f1

In [14]:
def model_classification_grid_search_svm_regression(model,x,y,cv,kernel,c,scaling,degree):
    
    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    try:
        param = {'C':np.arange(1,int(c)+1),'kernel':np.array([kernel]),'degree':np.arange(1,int(degree)+1)}
        grid = GridSearchCV(model,param_grid=param,cv=int(cv))
        grid.fit(x_train,y_train)
    except:
        param = {'C':np.arange(1,int(c)+1),'kernel':np.array(kernel),'degree':np.arange(1,int(degree)+1)}
        grid = GridSearchCV(model,param_grid=param,cv=int(cv))
        grid.fit(x_train,y_train)
    
    predict = grid.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test,predict))
    mse = mean_squared_error(y_test,predict)
    mae = mean_absolute_error(y_test,predict)
    r2,adj_r2 = r2_and_adjusted_r2(y_test,predict,x,y)
    
    return np.round(rmse,4),np.round(mse,4),np.round(mae,4),np.round(grid.best_score_,4),grid.best_params_,np.round(r2,4),np.round(adj_r2,4)

In [15]:
def model_classification_grid_search_svm_classification(model,x,y,cv,c,kernel,scaling,degree):

    x = pd.get_dummies(x)
    x = scaling.fit_transform(x)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=42)
    try:
        param = {'C':np.arange(1,int(c)+1),'kernel':np.array([kernel]),'degree':np.arange(1,int(degree)+1)}
        grid = GridSearchCV(model,param_grid=param,cv=int(cv))
        grid.fit(x_train,y_train)
    except:
        param = {'C':np.arange(1,int(c)+1),'kernel':np.array(kernel),'degree':np.arange(1,int(degree)+1)}
        grid = GridSearchCV(model,param_grid=param,cv=int(cv))
        grid.fit(x_train,y_train)
    predict_train = grid.predict(x_train)
    predict = grid.predict(x_test)
    acc_train = accuracy_score(y_train,predict_train)
    acc = accuracy_score(y_test,predict)
    precisionscore = precision_score(y_test,predict,average = None)
    recallscore = recall_score(y_test,predict,average = None)  
    roc_score = multiclass_roc_auc_score(y_test,predict)
    f1 = f1_score(y_test,predict,average=None)
    plotting_and_saving_confusion_matrix(y_test,predict)
    return np.round(acc_train,4),np.round(acc,4),precisionscore,recallscore,np.round(grid.best_score_,4),grid.best_params_,roc_score,f1

In [16]:
# def check_missing_values(df):
#     error = None
#     na_cols = df.isna().sum()
#     na_cols_list = []
#     for i in range(len(na_cols)):
#         if na_cols[i]>0:
#                 na_cols_list.append(na_cols.index[i])
#         if len(na_cols_list)>0:
#             error = "Missing Values Present. Please remove them"
#     return error

In [17]:
def replace_missing_values(df,value):
    if value == 'Mean':
        df = df.fillna(df.mean())
    elif value == 'Median':
        df = df.fillna(df.median())
    return df

In [18]:
@app.route('/linearregression',methods = ['GET','POST'])
def linearregression():
    error = None
    linear_types = ['Simple Regression','Ridge Regression (L2)', 'Lasso Regression (L1)']
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    linear_method = request.form.get('linear_types')
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        
        
        if linear_method =='Simple Regression':
            dependent_variable = request.form['dependent_variable']
            replacing = request.form['replace_types']
            scaling = request.form['scaling_types']
            df = replace_missing_values(df,replacing)
            #error = check_missing_values(df)
            cols = request.form.getlist('column')
            try:
                df = df.drop(cols,1)
            except:
                pass
            try:
                scaling = scaling_type_value(scaling)
                x = df.drop(dependent_variable,1)
                y = df[dependent_variable]
                rmse,mse,mae,r2,adj_r2 = model_regression(LinearRegression(),x,y,scaling)
                if rmse is not None:
                    try:
                        creating_report_regression('Linear Regression',rmse,mse,mae,r2,adj_r2,None,None)
                    except:
                        pass
                    return render_template('Linear Regression.html',linear_types = linear_types,columns = df.columns,scaling_types=scaling_types,replace_types=replace_types,rmse=rmse,mae=mae,mse=mse,r2=r2,adj_r2 = adj_r2)
            except:
                pass
            
            
            
        if linear_method == 'Ridge Regression (L2)':
            dependent_variable = request.form['dependent_variable']
            replacing = request.form['replace_types']
            scaling = request.form['scaling_types']
            #error = check_missing_values(df)
            cols = request.form.getlist('column')
            df = replace_missing_values(df,replacing)
            try:
                df = df.drop(cols,1)
            except:
                pass
            try:
                scaling = scaling_type_value(scaling)
                x = df.drop(dependent_variable,1)
                y = df[dependent_variable]
                rmse,mse,mae,r2,adj_r2 = model_regression(Ridge(),x,y,scaling)
                if rmse is not None:
                    try:
                        creating_report_regression('Ridge Regression',rmse,mse,mae,r2,adj_r2,None,None)
                    except:
                        pass
                    return render_template('Linear Regression.html',linear_types = linear_types,columns = df.columns,scaling_types=scaling_types,replace_types=replace_types,rmse=rmse,mae=mae,mse=mse,r2=r2,adj_r2 = adj_r2)
            except:
                pass
            
        if linear_method == 'Lasso Regression (L1)':
            dependent_variable = request.form['dependent_variable']
            replacing = request.form['replace_types']
            scaling = request.form['scaling_types']
            df = replace_missing_values(df,replacing)
            cols = request.form.getlist('column')
            try:
                df = df.drop(cols,1)
            except:
                pass
            try:
                scaling = scaling_type_value(scaling)
                x = df.drop(dependent_variable,1)
                y = df[dependent_variable]
                rmse,mse,mae,r2,adj_r2 = model_regression(Lasso(),x,y,scaling)
                if rmse is not None:
                    try:
                        creating_report_regression('Lasso Regression',rmse,mse,mae,r2,adj_r2,None,None)
                    except:
                        pass
                    return render_template('Linear Regression.html',linear_types = linear_types,columns = df.columns,scaling_types=scaling_types,replace_types=replace_types,rmse=rmse,mae=mae,mse=mse,r2=r2,adj_r2 = adj_r2)
            except:
                pass
            
    return render_template('Linear Regression.html',linear_types = linear_types,scaling_types=scaling_types,replace_types=replace_types,error=error,columns = df.columns)

In [19]:
@app.route('/logistic_regression',methods = ['GET','POST'])
def logistic_regression():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        
        
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,roc_score,f1= model_classification(LogisticRegression(),x,y,scaling)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    remove_report()
                    creating_report_classification('Logistic Regression',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,None,None)
                   
                except:
                    pass
                return render_template('Logistic Regression.html',acc_train=acc_train,replace_types=replace_types,scaling_types=scaling_types,acc=acc,pre=prec_score,rec=recal_score,roc=roc_score,f1=f1,columns = df.columns)
            
        except:
            pass
    return render_template('Logistic Regression.html',replace_types=replace_types,scaling_types=scaling_types,columns = df.columns,error = error)    

In [20]:
@app.route('/logistic_regression_hyperparameter',methods = ['GET','POST'])
def logistic_regression_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    regularizations = ['l1','l2']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        c = request.form['c']
        df = replace_missing_values(df,replacing)
        regularization = request.form.getlist('regular')
        
        if len(regularization)==0:
            error = 'Please select atleast 1 Kernel.'
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,best_score,best_param,roc_score,f1 = model_classification_grid_search_logistic(LogisticRegression(),x,y,c,cv,scaling,regularization)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    remove_report()
                    creating_report_classification('Logistic Regression with Hyper Parameters',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,best_score,best_param)
                except:
                    pass
                return render_template('Logistic Regression Hyperparameter.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,best_score = best_score,best_param = best_param,roc=roc_score,f1=f1,regularizations = regularizations)
            
        except:
            pass
    return render_template('Logistic Regression Hyperparameter.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,error = error,regularizations = regularizations)    

In [21]:
@app.route('/decision_tree_regression',methods = ['GET','POST'])
def decision_tree_regression():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'

        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,r2,adj_r2 = model_regression(DecisionTreeRegressor(),x,y,scaling)
            if rmse is not None:
                try:
                    creating_report_regression('Decision Tree Regression',rmse,mse,mae,r2,adj_r2,None,None)
                except:
                    pass
                return render_template('Decision Tree Regression.html',scaling_types = scaling_types,replace_types=replace_types,rmse=rmse,mae=mae,mse=mse,r2=r2,adj_r2 = adj_r2,columns = df.columns)
            
        except:
            pass
    return render_template('Decision Tree Regression.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [22]:
@app.route('/decision_tree_regression_hyperparameter',methods = ['GET','POST'])
def decision_tree_regression_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        max_depth = request.form['max_depth']
        min_samples_split = request.form['min_samples_split']
        min_samples_leaf = request.form['min_samples_leaf']
        min_weight_fraction_leaf = request.form['min_weight_fraction_leaf']
        max_leaf_nodes = request.form['max_leaf_nodes']
        df = replace_missing_values(df,replacing)
     
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,best_score,best_param,r2,adj_r2 = model_classification_grid_search_decision_tree_regression(DecisionTreeRegressor(),x,y,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_leaf_nodes,cv,scaling)
            if rmse is not None:
                try:
                    creating_report_regression('Decision Tree Regression with Hyperparameters',rmse,mse,mae,r2,adj_r2,best_score,best_param)
                except:
                    pass                
                return render_template('Decision Tree Regression Hyperparameter.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,rmse=rmse,mae=mae,mse=mse,error = error,best_score = best_score,best_param = best_param,r2=r2,adj_r2 = adj_r2)
            
        except:
            pass
    return render_template('Decision Tree Regression Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [23]:
@app.route('/decision_tree_classification',methods = ['GET','POST'])
def decision_tree_classification():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        

        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,roc_score,f1 = model_classification(DecisionTreeClassifier(),x,y,scaling)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    remove_report()
                    creating_report_classification('Decision Tree Classification',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,None,None)
                except:
                    pass
                return render_template('Decision Tree Classification.html',scaling_types=scaling_types,replace_types=replace_types,acc_train=acc_train,acc=acc,f1=f1,pre=prec_score,rec=recal_score,roc=roc_score,columns = df.columns)
            
        except:
            pass
    return render_template('Decision Tree Classification.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [24]:
@app.route('/decision_tree_classification_hyperparameter',methods = ['GET','POST'])
def decision_tree_classification_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        max_depth = request.form['max_depth']
        min_samples_split = request.form['min_samples_split']
        min_samples_leaf = request.form['min_samples_leaf']
        min_weight_fraction_leaf = request.form['min_weight_fraction_leaf']
        max_leaf_nodes = request.form['max_leaf_nodes']
        df = replace_missing_values(df,replacing)
        

        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,best_score,best_param,roc_score,f1 = model_classification_grid_search_decision_tree_classification(DecisionTreeClassifier(),x,y,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_leaf_nodes,cv,scaling)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    creating_report_classification('Decision Tree Classification with Hyperparameters',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,best_score,best_param)
                except:
                    pass
                return render_template('Decision Tree Classification Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,best_score = best_score,best_param = best_param,roc=roc_score,f1=f1)
        except:
            pass
    return render_template('Decision Tree Classification Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [25]:
@app.route('/random_forest_regression',methods = ['GET','POST'])
def random_forest_regression():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'
        
        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,r2,adj_r2 = model_regression(RandomForestRegressor(),x,y,scaling)
            if rmse is not None:
                try:
                    creating_report_regression('Random Forest Regression',rmse,mse,mae,r2,adj_r2,None,None)
                except:
                    pass
                return render_template('Random Forest Regression.html',scaling_types = scaling_types,replace_types=replace_types,rmse=rmse,mae=mae,mse=mse,columns = df.columns,r2=r2,adj_r2 = adj_r2)
            
        except:
            pass
    return render_template('Random Forest Regression.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [26]:
@app.route('/random_forest_regression_hyperparameter',methods = ['GET','POST'])
def random_forest_regression_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        max_depth = request.form['max_depth']
        min_samples_split = request.form['min_samples_split']
        min_samples_leaf = request.form['min_samples_leaf']
        min_weight_fraction_leaf = request.form['min_weight_fraction_leaf']
        max_leaf_nodes = request.form['max_leaf_nodes']
        df = replace_missing_values(df,replacing)
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,best_score,best_param,r2,adj_r2 = model_classification_grid_search_decision_tree_regression(RandomForestRegressor(),x,y,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_leaf_nodes,cv,scaling)
            if rmse is not None:
                try:
                    creating_report_regression('Random Forest Regression with Hyperparameters',rmse,mse,mae,r2,adj_r2,best_score,best_param)
                except:
                    pass 
                return render_template('Random Forest Regression Hyperparameter.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,rmse=rmse,mae=mae,mse=mse,error = error,best_score = best_score,best_param = best_param,r2=r2,adj_r2 = adj_r2)
            
        except:
            pass
               
    return render_template('Random Forest Regression Hyperparameter.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [27]:
@app.route('/random_forest_classification',methods = ['GET','POST'])
def random_forest_classification():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV Value not Numeric.'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,roc_score,f1 = model_classification(RandomForestClassifier(),x,y,scaling)
            
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    creating_report_classification('Random Forest Classification',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,None,None)
                except:
                    pass
                return render_template('Random Forest Classification.html',scaling_types = scaling_types,replace_types=replace_types,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,roc=roc_score,f1=f1,columns = df.columns)
            
        except:
            pass
    return render_template('Random Forest Classification.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [28]:
@app.route('/random_forest_classification_hyperparameter',methods = ['GET','POST'])
def random_forest_classification_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        max_depth = request.form['max_depth']
        min_samples_split = request.form['min_samples_split']
        min_samples_leaf = request.form['min_samples_leaf']
        min_weight_fraction_leaf = request.form['min_weight_fraction_leaf']
        max_leaf_nodes = request.form['max_leaf_nodes']
        
        df = replace_missing_values(df,replacing)
        
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,best_score,best_param,roc_score,f1 = model_classification_grid_search_decision_tree_classification(RandomForestClassifier(),x,y,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_leaf_nodes,cv,scaling)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    creating_report_classification('Random Forest Classification with Hyperparameters',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,best_score,best_param)
                except:
                    pass
                return render_template('Random Forest Classification Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,best_score = best_score,best_param = best_param,roc=roc_score,f1=f1)
        except:
            pass
    return render_template('Random Forest Classification Hyperparameter.html',scaling_types = scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [29]:
@app.route('/knn_regression',methods = ['GET','POST'])
def knn_regression():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'
        
        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,r2,adj_r2 = model_regression(KNeighborsRegressor(),x,y,scaling)
            
            if rmse is not None:
                try:
                    creating_report_regression('KNN Regression',rmse,mse,mae,r2,adj_r2,None,None)
                except:
                    pass                
                return render_template('KNN Regression.html',scaling_types = scaling_types,replace_types=replace_types,rmse=rmse,mae=mae,mse=mse,columns = df.columns,r2=r2,adj_r2 = adj_r2)
            
        except:
            pass
    return render_template('KNN Regression.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [30]:
@app.route('/knn_regression_hyperparameter',methods = ['GET','POST'])
def knn_regression_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    metrics = ['euclidean','manhattan','chebyshev','minkowski']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        neighbours = request.form['neighbours']
        metric = request.form.getlist('metric')
        df = replace_missing_values(df,replacing)
        
        if len(metric)==0:
            error = 'Please select atleast 1 Distance Metrics.'
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,best_score,best_param,r2,adj_r2 = model_classification_grid_search_knn_regression(KNeighborsRegressor(),x,y,cv,neighbours,metric,scaling) 
            
            if rmse is not None:
                try:
                    creating_report_regression('KNN Regression with Hyperparameters',rmse,mse,mae,r2,adj_r2,best_score,best_param)
                except:
                    pass 
                return render_template('KNN Regression Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,rmse=rmse,mae=mae,mse=mse,error = error,best_score = best_score,best_param = best_param,metrics=metrics,r2=r2,adj_r2 = adj_r2 )
            
        except:
            pass
    return render_template('KNN Regression Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error,metrics = metrics)    

In [31]:
@app.route('/knn_classification',methods = ['GET','POST'])
def knn_classification():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,roc_score,f1 = model_classification(KNeighborsClassifier(),x,y,scaling)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    remove_report()
                    creating_report_classification('KNN Classification',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,None,None)
                except:
                    pass
                return render_template('KNN Classification.html',scaling_types=scaling_types,replace_types=replace_types,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,roc=roc_score,f1=f1,columns = df.columns)
            
        except:
            pass
    return render_template('KNN Classification.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [32]:
@app.route('/knn_classification_hyperparameter',methods = ['GET','POST'])
def knn_classification_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    metrics = ['euclidean','manhattan','chebyshev','minkowski']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        neighbours = request.form['neighbours']
        metric = request.form.getlist('metric')
        df = replace_missing_values(df,replacing)
        if len(metric)==0:
            error = 'Please select atleast 1 Distance Metrics.'
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,best_score,best_param,roc_score,f1 = model_classification_grid_search_knn_classification(KNeighborsClassifier(),x,y,cv,neighbours,metric,scaling)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    creating_report_classification('KNN Classification with Hyperparameters',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,best_score,best_param)
                except:
                    pass
                return render_template('KNN Classification Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,best_score = best_score,best_param = best_param,metrics=metrics,roc=roc_score,f1=f1)
        except:
            pass
    return render_template('KNN Classification Hyperparameter.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error,metrics=metrics)    

In [33]:
@app.route('/svm_regression',methods=['GET','POST'])
def svm_regression():
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'
        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,r2,adj_r2 = model_regression(SVR(),x,y,scaling)
            if rmse is not None:
                try:
                    creating_report_regression('SVM Regression',rmse,mse,mae,r2,adj_r2,None,None)
                except:
                    pass                 
                return render_template('SVM Regression.html',scaling_types = scaling_types,replace_types=replace_types,rmse=rmse,mae=mae,mse=mse,columns = df.columns,r2=r2,adj_r2 = adj_r2)
            
        except:
            pass
    return render_template('SVM Regression.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [34]:
@app.route('/svm_regression_hyperparameter',methods = ['GET','POST'])
def svm_regression_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    kernel_types = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        kernel = request.form.getlist('kernel_type')
        cols = request.form.getlist('column')
        c = request.form['c']
        degree = request.form['degree']
        df = replace_missing_values(df,replacing)
        if len(kernel)==0:
            error = 'Please select atleast 1 Kernel.'
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            rmse,mse,mae,best_score,best_param,r2,adj_r2 = model_classification_grid_search_svm_regression(SVR(),x,y,cv,kernel,c,scaling,degree) 
            if rmse is not None:
                try:
                    creating_report_regression('SVM Regression with Hyperparameters',rmse,mse,mae,r2,adj_r2,best_score,best_param)
                except:
                    pass                
                return render_template('SVM Regression Hyperparameter.html',kernel_types = kernel_types,scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,rmse=rmse,mae=mae,mse=mse,error = error,best_score = best_score,best_param = best_param,r2=r2,adj_r2 = adj_r2)
            
        except:
            pass
    return render_template('SVM Regression Hyperparameter.html',kernel_types=kernel_types,scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [35]:
@app.route('/svm_classification',methods=['GET','POST'])
def svm_classification():
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        cols = request.form.getlist('column')
        df = replace_missing_values(df,replacing)
        
        
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
        except:
            error = 'Dependent Variable not present.'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,roc_score,f1 = model_classification(SVC(),x,y,scaling)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    creating_report_classification('SVM Classification',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,None,None)
                except:
                    pass
                return render_template('SVM Classification.html',scaling_types=scaling_types,replace_types=replace_types,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,roc=roc_score,f1=f1,columns = df.columns)
            
        except:
            pass
    return render_template('SVM Classification.html',scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [36]:
@app.route('/svm_classification_hyperparameter',methods = ['GET','POST'])
def svm_classification_hyperparameter():  
    error = None
    replace_types = ['Mean','Median']
    scaling_types = ['MinMaxScaler','StandardScaler','RobustScaler']
    kernel_types = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    if request.method == 'POST':
        dependent_variable = request.form['dependent_variable']
        cv = request.form['cv']
        replacing = request.form['replace_types']
        scaling = request.form['scaling_types']
        kernel = request.form.getlist('kernel_type')
        cols = request.form.getlist('column')
        c = request.form['c']
        degree = request.form['degree']
        df = replace_missing_values(df,replacing)
        
        if len(kernel)==0:
            error = 'Please select atleast 1 Kernel.'
        try:
            df = df.drop(cols,1)
        except:
            pass
        #error = check_missing_values(df)
        try:
            value = df[dependent_variable].value_counts().tolist()
            x = df.drop(dependent_variable,1)
            y = df[dependent_variable]
            cv = int(cv)
        except:
            error = 'Dependent Variable not present / CV value not Numeric'
        try:
            scaling = scaling_type_value(scaling)
            acc_train,acc,prec_score,recal_score,best_score,best_param,roc_score,f1 = model_classification_grid_search_svm_classification(SVC(),x,y,cv,c,kernel,scaling,degree)
            if acc is not None:
                try:
                    solution = checking_problem(value,acc_train,acc,prec_score,recal_score)
                    creating_report_classification('SVM Classification with Hyperparameters',solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,best_score,best_param)
                except:
                    pass
                return render_template('SVM Classification Hyperparameter.html',kernel_types = kernel_types,scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,acc_train=acc_train,acc=acc,pre=prec_score,rec=recal_score,best_score = best_score,best_param = best_param,roc=roc_score,f1=f1)
        except:
            pass
    return render_template('SVM Classification Hyperparameter.html',kernel_types=kernel_types,scaling_types=scaling_types,replace_types=replace_types,columns = df.columns,error = error)    

In [37]:
@app.route('/uploader', methods = ['GET', 'POST'])
def upload_file():
    removal()
    if request.method == 'POST':
        f = request.files['file']
        f.save(secure_filename(f.filename))
    return render_template('welcome.html',message = 'File Uploaded')

In [38]:
@app.route('/remove_datasets',methods = ['GET','POST'])
def removal():
    if request.method == 'POST':
        files = glob.glob('*.csv')
        for i in files:
            os.remove(i)
            
        files = glob.glob('*.docx')
        for i in files:
            os.remove(i)
            
        files = glob.glob('*.PNG')
        for i in files:
            os.remove(i)
            
    return render_template('welcome.html',message = 'Your CSV has been deleted')

In [39]:
def graph(cols_1,cols_2,x_coordinates, y_coordinates):
    sns.set(rc={'figure.figsize':(11.7,8.27)})
    plot = sns.swarmplot(x_coordinates, y_coordinates)
    fig = plot.get_figure()
    fig.savefig(cols_1+'_'+cols_2+'.PNG')
    

In [40]:
def build_graph(x_coordinates, y_coordinates):
    img = io.BytesIO()
    plot = sns.swarmplot(x_coordinates, y_coordinates)
    #plt.scatter(x_coordinates, y_coordinates)
    #plt.savefig(img, format='png')
    
    fig = plot.get_figure()
    fig.savefig(img, format='png')
    img.seek(0)
    graph_url = base64.b64encode(img.getvalue()).decode()
    #plt.close()
    return 'data:image/png;base64,{}'.format(graph_url)

In [41]:
@app.route('/visualizations',methods = ['GET','POST'])
def visualizations():
    error = None
    df = reading_csv()
    if type(df)==str:
        return render_template('welcome.html',message = 'Please upload a CSV first')
    
    if request.method == 'POST':
        cols = request.form.getlist('column')
        graph(cols[0],cols[1],df[cols[0]],df[cols[1]])

#         if graph1_url is not None:
#             return render_template('Visualizations.html',graph1=graph1_url,columns = df.columns) 
    
    return render_template('Visualizations.html',columns=df.columns,error = error)

In [42]:
def reading_solutions_excel():
    excels = glob.glob('*.xlsx')
    excel_df = pd.read_excel(excels[0])
    print(excels)
    return excel_df

In [43]:
sol = ['Your Dataset might be imbalanced. You can try different techniques of fixing that, like 1) Using SMOTE 2) Using different Class Weights 3) Try Decision tree or Random Forest.',
'Your Recall is very low for few classes. Please check if the dataset is balanced or not.',
'Precision is low for few classes as there might be a possibility that the dataset is imbalanced.',
'Your Training accuracy is much higher than your testing accuracy as model might be overfitting. Please try and tune your model or try another algorithm. Also, you can try some regularizations.',
'Your Testing accuracy is a bit higher than your training accuracy as your model might be underfitting. Please try more complex algorithms or hyper tune this model.']
excels_df = pd.DataFrame({'Condition':['Imbalance','Low Recall','Low Precision','Overfitting','Underfitting'],'Solution':sol})

In [44]:
def checking_problem(value,acc_train,acc,prec_score,recal_score):
    #excels_df = reading_solutions_excel()
    #excels_df = excels_df1
    solutions = []
    conditions = []
#     for i in range(len(value)):
#         for j in range(i+1,len(value)):
#             if value[i]<0.6*value[j] or value[j]<0.6*value[i]:
    if min(value)<0.6*max(value):
        conditions.append('Imbalance')
        
    if min(recal_score)<0.6*max(recal_score):
        conditions.append('Low Recall')
        
    if min(prec_score)<0.6*max(prec_score):
        conditions.append('Low Precision')
      
    if acc-acc_train>0.03:
        conditions.append('Underfitting')
        
    if acc_train-acc>0.03:
        conditions.append('Overfitting')
                
    if 'Low Recall' in conditions:
        for i in range(len(excels_df)):
            if excels_df['Condition'][i]=='Low Recall':
                solutions.append(excels_df['Solution'][i])
                 
    if 'Low Precision' in conditions:
        for i in range(len(excels_df)):
            if excels_df['Condition'][i]=='Low Precision':
                solutions.append(excels_df['Solution'][i])
                
    if 'Imbalance' in conditions:
        for i in range(len(excels_df)):
            if excels_df['Condition'][i]=='Imbalance':
                solutions.append(excels_df['Solution'][i])
                
    if 'Overfitting' in conditions:
        for i in range(len(excels_df)):
            if excels_df['Condition'][i]=='Overfitting':
                solutions.append(excels_df['Solution'][i])
                
    if 'Underfitting' in conditions:
        for i in range(len(excels_df)):
            if excels_df['Condition'][i]=='Underfitting':
                solutions.append(excels_df['Solution'][i])
                
    solutions_1 = list(set(solutions))
    return ' '.join(solutions_1)      
    
    #return None

In [45]:
def creating_report_regression(model_name,rmse,mse,mae,r2,adj_r2,best_score,best_param):
    
    document.add_heading('Report for '+model_name, 0)
    table = document.add_table(rows=1, cols=5,style='Light Shading Accent 2')
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Root Mean Square Error'
    hdr_cells[1].text = 'Mean Square Error'
    hdr_cells[2].text = 'Mean Absolute Error'
    hdr_cells[3].text = 'R Square'
    hdr_cells[4].text = 'Adjusted R Square'
    
    row_cells = table.add_row().cells
    row_cells[0].text = str(rmse)
    row_cells[1].text = str(mse)
    row_cells[2].text = str(mae)
    row_cells[3].text = str(r2)
    row_cells[4].text = str(adj_r2)   
    
    p = document.add_paragraph(' ')
    
    if best_score is not None:
        table = document.add_table(rows=1, cols=2,style='Light Shading Accent 6')
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'Best Score'
        hdr_cells[1].text = 'Best Parameters'
        row_cells = table.add_row().cells
        row_cells[0].text = str(np.round(best_score*100,2))+'%'
        row_cells[1].text = str(best_param)
    
    
    p = document.add_paragraph(' ')
    
    document.save('Report.docx',)

In [46]:
def creating_report_classification(model_name,solution,acc_train,acc,prec_score,recal_score,roc_score,f1,value,best_score,best_param):
    
    document.add_heading('Report for '+model_name, 0)
    if len(solution)>0:
        p = document.add_paragraph('Suggestion: '+solution)
    else:
        p = document.add_paragraph('Suggestion: Everything looks good from our side.')
    
    
    if roc_score is not None:
        p = document.add_heading('ROC AUC Score: '+str(roc_score),1)
        p = document.add_paragraph(' ')
     
    table = document.add_table(rows=1, cols=2,style='Light Shading Accent 2')
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Training Accuracy'
    hdr_cells[1].text = 'Testing Accuracy'
    row_cells = table.add_row().cells
    row_cells[0].text = str(np.round(acc_train*100,2))+'%'
    row_cells[1].text = str(np.round(acc*100,2))+'%'
    
    p = document.add_paragraph(' ')
     
    table = document.add_table(rows=1, cols=5,style='Light Shading Accent 4')
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Class Name'
    hdr_cells[1].text = 'Value Count'
    hdr_cells[2].text = 'Precision Score'
    hdr_cells[3].text = 'Recall Score'
    hdr_cells[4].text = 'F1 Score'
    
    for i in range(len(prec_score)):
        row_cells = table.add_row().cells
        row_cells[0].text = 'Class '+str(i)
        row_cells[1].text = str(value[i])
        row_cells[2].text = str(np.round(prec_score[i],5))
        row_cells[3].text = str(np.round(recal_score[i],5))
        row_cells[4].text = str(np.round(f1[i],5))

    p = document.add_paragraph(' ')    
    
    if best_score is not None:
        table = document.add_table(rows=1, cols=2,style='Light Shading Accent 6')
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'Best Score'
        hdr_cells[1].text = 'Best Parameters'
        row_cells = table.add_row().cells
        row_cells[0].text = str(np.round(best_score*100,2))+'%'
        row_cells[1].text = str(best_param)
    
    p = document.add_paragraph(' ')
    
    try:
        graphs = glob.glob('*.PNG')
        for i in range(len(graphs)):
            document.add_picture(graphs[i], width=Inches(4))
    except:
        pass
    
    document.save('Report.docx',)

In [47]:
@app.route('/download_report',methods = ['GET','POST'])
def download_report():
    path = 'Report.docx'
    if request.method == 'GET':
        path = 'Report.docx'
    if request.method == 'POST':
        path = "Report.docx"
    return send_file(path, as_attachment=True)

In [48]:
def remove_report():
    files = glob.glob('*.docx')
    for i in files:
        os.remove(i)

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/linear.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/logistic.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/decision%20tree%20regression.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/knnr.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/dd.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/knnc.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/rf%20classification.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/rf%20regression.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/svc.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/visualizations.PNG HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2019 08:16:33] "GET /static/svr.PNG HTTP/1.1"